## Step 13: 가변 길이 인수(역전파 편)

앞서 순전파를 가변 길이 인수에 대응할 수 있도록 수정하였다. 이번에는 역전파 부분에서 가변 길이 인수를 처리할 수 있도록 해보겠다.

### 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파

덧셈 계산에서, 순전파는 입력이 두 개, 출력이 한 개이다. 이에 따라 역전파에서는 입력이 한 개, 출력이 두 개가 된다.

덧셈의 역전파는 출력 쪽에서 전해지는 미분값에 1을 곱한 값이 입력 변수 (x0, x1)의 미분이다. 즉, 상류에서 흘러오는 미분값을 그대로 흘려보내는 것이 덧셈의 역전파인 셈이다.

```python
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy
```

이와 같이 backward에서 입력 1개, 출력 2개를 갖기 위해서는 역전파의 핵심 구현을 바꿔야 한다. 여기서는 Variable 클래스의 backward 메서드를 수정하도록 하자.

### 13.2 Variable 클래스 수정

```python
class Variable:
    ...
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)
```

기존의 Variable 클래스는 while 블록 안에서 함수의 입출력을 얻고 backward 메서드를 호출한다. 이는 우리가 지금까지는 입출력이 각각 하나씩이라고 가정했기 때문이다. 이를 수정하도록 하자.

```python
class Variable:
    ...
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] # 1
            gxs = f.backward(*gys) # 2
            if not isinstance(gxs, tuple): # 3
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # 4
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)
```

총 4군데를 수정하였다. 우선 # 1에서 출력 변수인 outputs에 담겨 있는 미분값들을 리스트에 담도록 하였다. # 2에서 함수 f의 역전파를 호출하는데 이때 `f.backward(*gys)`처럼 리스트 언팩을 수행해준다. # 3에서는 gxs가 튜플이 아닐 경우 튜플로 변환하는 과정을 거쳤다.

마지막으로 # 4에서 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장해둔다. 여기서 gxs와 f.inputs의 각 원소는 zip 함수와 for 문에 의해 서로 대응 관계에 있게 된다. f.inputs[i]의 미분값이 gxs[i]에 대응한다는 뜻이다.

### 13.3 Square 클래스 구현

Add 함수를 구현한 방법과 동일하게, Square 함수도 구현해보도록 하겠다.

```python
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 수정 전: x = self.input.data
        gx = 2 * x * gy
        return gx
```

Function 클래스의 인스턴스 변수 이름을 input에서 inputs로 변경하여 가변 길이 인수를 처리한다는 느낌을 더했다. 여기서 입력 변수 x를 가져오도록만 수정해주면 끝이다.

```python
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data) # 2^2 + 3^2 = 13.0
print(x.grad) # 2^2 = 4.0
print(y.grad) # (y^2)' = 2y / 2 * 3 = 6.0 
```

$z = x^2 + y^2$이라는 계산을 `z = add(square(x), square(y))`라는 코드로 풀어냈다. 이제 z.backward()를 호출하기만 하면 자동으로 미분 계산이 수행된다.

이로써 복수 입출력에 대응하는 자동 미분 구조를 완성했다. 이제 더 복잡한 함수도 같은 방식으로 구현하면 될 것 같다. 하지만 아직도 여기에는 문제점이 숨어있다. 이를 다음 단계에서 알아보고 해결하도록 하겠다.